In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

from tqdm import tqdm

out_dir = "../../output/count"
os.makedirs(out_dir, exist_ok=True)


In [2]:
airport = "kmdw"
base_dir = '../../datasets/amelia'
traj_version = 'a10v08'



traj_dir = os.path.join(base_dir, f'traj_data_{traj_version}', 'raw_trajectories', f'{airport}')

traj_files = [os.path.join(traj_dir, f) for f in os.listdir(traj_dir)]
print(traj_files[0])

data = pd.read_csv(traj_files[0])
data.head()

../../datasets/amelia/traj_data_a10v08/raw_trajectories/kmdw/KMDW_184_1686178800.csv


,Frame,ID,Altitude,Speed,Heading,Lat,Lon,Range,Bearing,Type,Interp,x,y
0,0,2063735,612.5,0.0,94.130859,41.782290,-87.751580,0.491627,3.024479,1.0,[ORG],-0.488260,0.057445
1,0,2065063,612.5,3.1,140.601929,41.783812,-87.744674,0.707620,2.038758,0.0,[ORG],-0.319185,0.631543
2,0,2065188,612.5,0.0,358.571777,41.791940,-87.746520,0.754364,0.686298,2.0,[ORG],0.583574,0.478024
3,0,2064899,612.5,0.7,44.477304,41.788320,-87.760220,0.685238,-1.302699,2.0,[ORG],0.181518,-0.660759
4,0,2064965,612.5,8.3,190.194214,41.781674,-87.753834,0.571641,-2.912282,0.0,[ORG],-0.556677,-0.129937


In [3]:
agent_counts = {
    'total': [], '0': [], '1': [], '2': []
}
total_count = 0
for f in tqdm(traj_files):
    data = pd.read_csv(f)

    unique_frames = np.unique(data.Frame)
    for frame in unique_frames:
        agents = data[data.Frame == frame]

        agent_counts['total'].append(agents.shape[0])
        agent_counts['0'].append(np.where(agents.Type == 0.0)[0].shape[0])
        agent_counts['1'].append(np.where(agents.Type == 1.0)[0].shape[0])
        agent_counts['2'].append(np.where(agents.Type == 2.0)[0].shape[0])


100%|██████████| 100/100 [01:25<00:00,  1.17it/s]


In [4]:
agent_count_stats = {}
for k, v in agent_counts.items():
    v = np.asarray(v)
    agent_count_stats[k] = {
        "min": round(v.min().astype(float), 5),
        "max": round(v.max().astype(float), 5),
        "mean": round(v.mean().astype(float), 5),
        "std": round(v.std().astype(float), 5)
    }

    agent_counts[k] = v

agent_count_stats

{'total': {'min': 2.0, 'max': 35.0, 'mean': 11.34282, 'std': 4.88353},
 '0': {'min': 0.0, 'max': 15.0, 'mean': 2.63311, 'std': 2.4362},
 '1': {'min': 0.0, 'max': 19.0, 'mean': 2.67156, 'std': 3.86028},
 '2': {'min': 2.0, 'max': 18.0, 'mean': 6.03808, 'std': 1.98603}}

In [5]:
with open(f'{out_dir}/{airport}_stats.json', 'w') as f:
    json.dump(agent_count_stats, f, indent=2)

with open(f'{out_dir}/{airport}.pkl', 'wb') as handle:
    pickle.dump(agent_counts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
name = {'total': 'Total', '0': 'Aircraft', '1': 'Vehicle', '2': 'Unknown'}
colors = {'ksea': 'blue', 'kewr': 'red', 'kbos': 'green', 'kmdw': 'orange'}
for key, value in name.items():
    arr = agent_counts[key]

    plt.hist(arr, bins = int(arr.max()), color=colors[airport])
    plt.title(f"{value}")
    plt.xlabel("Number of Agents per Timestep")
    plt.ylabel("Count")
    plt.savefig(f"{out_dir}/{airport}_{key}.png", dpi=600)
    plt.close()

In [7]:
for key, value in name.items():
    arr = agent_counts[key]
    q_lower = np.quantile(arr, 0.05)
    q_upper = np.quantile(arr, 0.95)

    arr = arr[(arr >= q_lower) & (arr <= q_upper)]

    plt.hist(arr, bins = int(arr.max()), color=colors[airport])
    plt.title(f"{value}")
    plt.xlabel("Number of Agents per Timestep")
    plt.ylabel("Count")
    plt.savefig(f"{out_dir}/{airport}_{key}_iqr.png", dpi=600)
    plt.close()